In [ ]:
# Determine the type of network to use
NETWORK_TYPE = "railway"  # "road" or "railway" for rail networks

## 1. Import Road Data
Two options:
 - from the `data` folder read the file corresponding to the desired road network on which to build the buffers.
- fetch road or rail network data (from OpenStreetMap)

In [ ]:
import geopandas as gpd
roads_gdf = gpd.read_file("../data/road_networks/A9.gpkg")
rail_gdf = gpd.read_file("../data/railway_networks/Lausanne_rail_100_150.gpkg")

## 2. Generate Circular Sampling Zones around the Network
Generates a geodataframe of buffers around the road or rail network, and creates an HTML file to visualize the buffers.


In [ ]:
RADIUS = 150
STEP = 300
TOL = 150  # tolerance for the buffer simplification
name = "vaud_railway"  # name of the network, used for saving files

In [ ]:
from src.buffers.buffers_generation import make_buffers

buffers_gdf = make_buffers(rail_gdf, name=name, step=STEP, radius=RADIUS, tol=TOL)

In [ ]:
from src.buffers.buffers_visualization import visualize_buffers

visualize_buffers(buffers_gdf, html_filename=f"{name}_buffer_{RADIUS}_{STEP}.html")

## 3. Add Data to Sampling Zones

### Habitats Data
Gets the habitat data and computes the intersection with each buffer, which includes the area of each habitat type within the buffer and the percentage of the buffer area that is covered by that habitat type.
**This computation can take a while (around 30 mins).**

In [ ]:
from src.buffers.assign_data_to_buffers import assign_habitats_to_buffers

habitats_gdf = gpd.read_file("../data/habitats/habitat_map_vaud.gpkg")

In [ ]:
target_crs = 2056
habitats_gdf = habitats_gdf.to_crs(target_crs)
buffers_gdf = buffers_gdf.to_crs(target_crs)

In [ ]:
habitats_in_buf_df = assign_habitats_to_buffers(habitats_gdf, buffers_gdf)

In [ ]:
# Add columns for buffer IDs
buffers_gdf = buffers_gdf.reset_index(drop=True)
buffers_gdf['buffer_id'] = buffers_gdf.index + 1

In [ ]:
# Save files
buffers_gdf.to_file(f"../data/buffers/{name}_buffers_{RADIUS}_{STEP}_{TOL}.geojson", driver="GeoJSON")
habitats_in_buf_df.to_csv(f"../data/habitats/habitats_in_{name}_buffers.csv", index=False)

### For Roads:

In [ ]:
from src.buffers.assign_data_to_buffers import assign_road_data_to_buffers

if NETWORK_TYPE == "road":
    roads_df = assign_road_data_to_buffers(roads_gdf, buffers_gdf)
    # Save the road data to a CSV file
    # roads_df.to_csv(f"../data/road_networks/road_data_in_{name}_buffers.csv", index=False)
    print(roads_df.columns)

### For Railway:

In [ ]:
from src.buffers.assign_data_to_buffers import assign_rail_data_to_buffers

if NETWORK_TYPE == "railway":
    rail_df = assign_rail_data_to_buffers(rail_gdf, buffers_gdf)
    # Save the rail data to a CSV file
    rail_df.to_csv(f"../data/railway_networks/rail_data_in_{name}_buffers.csv", index=False)

In [ ]:
print(habitats_in_buf_df.info)